<a href="https://colab.research.google.com/github/MidsummerYU/Project_RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pypdf python-dotenv  # extraire texte depuis pdf
!pip install transformers
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index  # data parsing
!%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"

/bin/bash: line 1: fg: no job control


In [2]:
# For the text splitter
import pandas as pd

# for the model:  https://huggingface.co/databricks/dolly-v2-3b
import torch
from transformers import pipeline

**Test with the model only**

In [3]:

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
res = generate_text("what happened on March 12th 2024?")
print(res[0]["generated_text"])

On March 12th, 2024, a wildfire broke out in the Cascade Range in Oregon, United States, threatening over 36,000 people' homes. This large fire had been burning for days, feeding on dry and windsy conditions. The day before the fire started was notably dry.


**Creation of our RAG**

In [5]:
import pandas as pd

file_path = 'export-bordeaux202401.csv'
column_names = ['DATE', 'MAX_TEMPERATURE_C', 'MIN_TEMPERATURE_C']

df = pd.read_csv(file_path, skiprows=3, usecols=column_names, names=column_names)

# Exclude the first element (column name) from each column
for col in df.columns:
    df[col] = df[col].iloc[1:]

# Drop the first row (index 0) from the DataFrame
df = df.drop(index=0)

# Convert the DataFrame to a dictionary without column names as keys
weather_data = df.to_dict(orient='list')

# Display the first few elements of the dictionary
print(weather_data)

{'DATE': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12', '2024-01-13', '2024-01-14', '2024-01-15', '2024-01-16', '2024-01-17', '2024-01-18', '2024-01-19', '2024-01-20', '2024-01-21', '2024-01-22', '2024-01-23', '2024-01-24', '2024-01-25', '2024-01-26', '2024-01-27', '2024-01-28', '2024-01-29', '2024-01-30', '2024-01-31'], 'MAX_TEMPERATURE_C': ['9', '11', '13', '12', '10', '8', '6', '4', '4', '3', '4', '5', '8', '10', '8', '10', '13', '13', '5', '6', '6', '10', '13', '13', '16', '13', '16', '15', '15', '15', '16'], 'MIN_TEMPERATURE_C': ['6', '6', '9', '7', '6', '3', '3', '-2', '-3', '0', '2', '1', '0', '3', '5', '4', '11', '3', '0', '-2', '0', '3', '6', '11', '9', '7', '8', '8', '8', '8', '8']}


In [5]:
# Convert 'DATE' column to datetime type
df['DATE'] = pd.to_datetime(df['DATE'])

# Set 'DATE' column as the index of the DataFrame
df.set_index('DATE', inplace=True)

**Test with the RAG**

In [ ]:
def get_weather_info(query, weather_data):
    # Perform question answering using LangChain pipeline
    answer = pipeline('question-answering', model="databricks/dolly-v2-3b")(question=query, context=weather_data)
    return answer

# Example usage
query = "What was the weather on March 12th, 2024?"

# Retrieve weather information based on the user query
response = get_weather_info(query, weather_data)
print(response['answer'])